`Дисциплина: Методы и технологии машинного обучения`   
`Уровень подготовки: бакалавриат`   
`Направление подготовки: 01.03.02 Прикладная математика и информатика`   
  




# Исследование факторов, повышающих вероятность происшествия ДТП со смертельным исходом в Московской области

### Проект выполнила Бобина Татьяна Сергеевна

*Данные*: официальный сайт ГИБДД, но данные приведены с изменениями (например, корректировки координат)

In [1]:
import geopandas as gpd

import re
import nltk
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import sleep
from random import randint

from bs4 import BeautifulSoup
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from nltk.probability import FreqDist
from sklearn.decomposition import PCA
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import pymorphy2

nltk.download('punkt')
nltk.download('stopwords')

True

### Скачиваем набор данных и проверяем корректность загрузки

In [2]:
df = gpd.read_file(r"moskovskaia-oblast.geojson")

In [3]:
df

,id,tags,light,point,nearby,region,scheme,address,weather,category,...,severity,vehicles,dead_count,participants,injured_count,parent_region,road_conditions,participants_count,participant_categories,geometry
0,2152021,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено","{'lat': 55.870062, 'long': 37.993727}",[Жилые дома индивидуальной застройки],Щелковский,070,"д Медвежьи Озера, А-103 Щелковское шоссе Москв...",[Пасмурно],Столкновение,...,Легкий,"[ { ""year"": 2018, ""brand"": ""HYUNDAI"", ""color"":...",0,[ ],1,Московская область,[Сухое],4,[Все участники],POINT (37.99373 55.87006)
1,2144587,[Дорожно-транспортные происшествия],Светлое время суток,"{'lat': 55.868966, 'long': 37.992911}","[Многоквартирные жилые дома, Жилые дома индиви...",Щелковский,740,"д Медвежьи Озера, А-103 Щелковское шоссе Москв...",[Пасмурно],Наезд на пешехода,...,Тяжёлый,"[ { ""year"": 2006, ""brand"": ""SKODA"", ""color"": ""...",0,"[ { ""role"": ""Пешеход"", ""gender"": ""Женский"", ""v...",1,Московская область,[Сухое],2,"[Пешеходы, Все участники]",POINT (37.99291 55.86897)
2,2160233,[Дорожно-транспортные происшествия],"В темное время суток, освещение отсутствует","{'lat': 56.071667, 'long': 38.293333}",[],Щелковский,600,"ЩЕЛКОВО-ФРЯНОВО, 31 км",[Снегопад],Опрокидывание,...,Тяжёлый,"[ { ""year"": 2011, ""brand"": ""HYUNDAI"", ""color"":...",0,[ ],2,Московская область,[Заснеженное],3,[Все участники],POINT (38.29333 56.07167)
3,2165667,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено","{'lat': 55.880992, 'long': 38.003705}",[Регулируемый перекресток],Щелковский,300,А-103 Щелковское шоссе Москва - Щелково - авто...,"[Пасмурно, Дождь]",Столкновение,...,Легкий,"[ { ""year"": 2012, ""brand"": ""ВАЗ"", ""color"": ""Се...",0,[ ],1,Московская область,[Мокрое],2,[Все участники],POINT (38.00370 55.88099)
4,2501444,[Дорожно-транспортные происшествия],Светлое время суток,"{'lat': 55.859117, 'long': 37.983395}",[Надземный пешеходный переход],Щелковский,200,"д Долгое Ледово, А-103 Щелковское шоссе Москва...",[Ясно],Столкновение,...,Легкий,"[ { ""year"": 2014, ""brand"": ""SKODA"", ""color"": ""...",0,[ ],1,Московская область,[Пыльное],2,[Все участники],POINT (37.98340 55.85912)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47114,2135111,[Дорожно-транспортные происшествия],Светлое время суток,"{'lat': 55.8027, 'long': 38.0193}","[Жилые дома индивидуальной застройки, Остановк...",Балашиха,070,"г Балашиха, М-7 Волга Москва - Владимир - Нижн...",[Ясно],Столкновение,...,Легкий,"[ { ""year"": 2013, ""brand"": ""ГАЗ"", ""color"": ""Бе...",0,[ ],1,Московская область,[Сухое],3,[Все участники],POINT (38.01930 55.80270)
47115,2135154,[Дорожно-транспортные происшествия],Светлое время суток,"{'lat': 55.0689, 'long': 37.0364}","[Внутридворовая территория, Лечебные учреждения]",Балашиха,880,"г Балашиха, пр-кт Ленина, 63",[Пасмурно],Наезд на пешехода,...,Тяжёлый,"[ { ""year"": 2010, ""brand"": ""NISSAN"", ""color"": ...",0,"[ { ""role"": ""Пешеход"", ""gender"": ""Женский"", ""v...",1,Московская область,"[Недостатки зимнего содержания, Со снежным нак...",2,"[Пешеходы, Все участники]",POINT (37.03640 55.06890)
47116,2148619,"[Дорожно-транспортные происшествия, ДТП и пост...",Светлое время суток,"{'lat': 55.4729, 'long': 37.5555}","[Многоквартирные жилые дома, Остановка обществ...",Балашиха,740,"г Балашиха, ул Разина, 8",[Ясно],Наезд на велосипедиста,...,Тяжёлый,"[ { ""year"": 2008, ""brand"": ""TRABANT"", ""color"":...",0,[ ],1,Московская область,"[Сухое, Неудовлетворительное состояние обочин]",2,"[Велосипедисты, Дети, Все участники]",POINT (37.55550 55.47290)
47117,2148630,[Дорожно-транспортные происшествия],Светлое время суток,"{'lat': 55.46, 'long': 37.548}","[Жилые дома индивидуальной застройки, Нерегули...",Балашиха,500,"г Балашиха, Балашиха-Салтыковка-ст. Никольская...",[Ясно],Столкновение,...,Легкий,"[ { ""year"": 2015, ""brand"": ""KIA"", ""color"": ""Бе...",0,[ ],1,Московская обл

In [3]:
# Вывод типов данных

In [4]:
df.dtypes

id                                 int64
tags                              object
light                             object
point                             object
nearby                            object
region                            object
scheme                            object
address                           object
weather                           object
category                          object
datetime                  datetime64[ns]
severity                          object
vehicles                          object
dead_count                         int64
participants                      object
injured_count                      int64
parent_region                     object
road_conditions                   object
participants_count                 int64
participant_categories            object
geometry                        geometry
dtype: object

In [2]:
# Проверим на пропуски

In [5]:
df.isna().sum()

id                           0
tags                         0
light                        0
point                        0
nearby                       0
region                       0
scheme                    1397
address                   1623
weather                      0
category                     0
datetime                     0
severity                     0
vehicles                     0
dead_count                   0
participants                 0
injured_count                0
parent_region                0
road_conditions              0
participants_count           0
participant_categories       0
geometry                     3
dtype: int64

# Выделяем нужные колонки

In [6]:
df_remade = df[['severity', 'participants_count', 'region', 'participant_categories', 'category', 'datetime', 'light', 'weather', 'road_conditions']]
# 'dead_count', 'injured_count', 'nearby', 'address', 
# df = df[['dead_count', 'light', 'weather', 'road_conditions', 'category', 'severity', 'vehicles', 'participants']]
df_remade

,severity,participants_count,region,participant_categories,category,datetime,light,weather,road_conditions
0,Легкий,4,Щелковский,[Все участники],Столкновение,2019-11-17 17:20:00,"В темное время суток, освещение включено",[Пасмурно],[Сухое]
1,Тяжёлый,2,Щелковский,"[Пешеходы, Все участники]",Наезд на пешехода,2018-11-05 07:35:00,Светлое время суток,[Пасмурно],[Сухое]
2,Тяжёлый,3,Щелковский,[Все участники],Опрокидывание,2015-11-22 23:45:00,"В темное время суток, освещение отсутствует",[Снегопад],[Заснеженное]
3,Легкий,2,Щелковский,[Все участники],Столкновение,2019-10-10 00:55:00,"В темное время суток, освещение включено","[Пасмурно, Дождь]",[Мокрое]
4,Легкий,2,Щелковский,[Все участники],Столкновение,2020-09-25 15:15:00,Светлое время суток,[Ясно],[Пыльное]
...,...,...,...,...,...,...,...,...,...
47114,Легкий,3,Балашиха,[Все участники],Столкновение,2016-02-17 14:30:00,Светлое время суток,[Ясно],[Сухое]
47115,Тяжёлый,2,Балашиха,"[Пешеходы, Все участники]",Наезд на пешехода,2016-02-03 16:20:00,Светлое время суток,[Пасмурно],"[Недостатки зимнего содержания, Со снежным нак..."
47116,Тяжёлый,2,Балашиха,"[Велосипедисты, Дети, Все участники]",Наезд на велосипедиста,2017-04-18 17:19:00,Светлое время суток,[Ясно],"[Сухое, Неудовлетворительное состояние обочин]"
47117,Легкий,3,Балашиха,[Все участники],Столкновение,2017-04-29 20:00:00,Светлое время суток,[Ясно],"[Сухое, Отсутствие, плохая различимость горизо..."


# Преобразуем значения в столбцах

In [7]:
df_transform = df_remade.copy()

In [8]:
# severity               —————
# participants_count     —————
# dead_count             —————
# injured_count          —————
# region                 + заменить на значения ✔
# participant_categories преобразовать 0 и 1 ✔
# category               —————  ✔
# datetime               преобразовать разделив промежутки на час ✔
# light                  преобразовать ✔
# weather                преобразовать ✔
# road_conditions        преобразовать ✔

In [9]:
# Заменяем категориальные значения на числовые
light_dict = {
    'Светлое время суток': 0,
    'В темное время суток, освещение включено': 1,
    'В темное время суток, освещение отсутствует': 2,
    'В темное время суток, освещение не включено': 3,
    'Сумерки': 4
}

severity_dict = {
    'Легкий': 0,
    'Тяжёлый': 0,
    'С погибшими': 1
}

df_transform['light'] = df_transform['light'].replace(light_dict)
df_transform['severity'] = df_transform['severity'].replace(severity_dict)

In [10]:
# participant_categories
# Преобразование колонки participant_categories с использованием метода get_dummies
participant_categories_dummies = pd.get_dummies(df_transform.participant_categories.apply(pd.Series).stack())
participant_categories_dummies.index = participant_categories_dummies.index.droplevel(-1)
participant_categories_dummies = participant_categories_dummies.groupby(participant_categories_dummies.index).sum()

# Объединение исходного датафрейма с преобразованными колонками
df_transform = pd.concat([df_transform, participant_categories_dummies], axis=1)

# Удаление старой колонки participant_categories
df_transform = df_transform.drop('participant_categories', axis=1)

In [11]:
# weather
# Преобразование колонки weather с использованием метода get_dummies
weather_dummies = pd.get_dummies(df_transform.weather.apply(pd.Series).stack())
weather_dummies.index = weather_dummies.index.droplevel(-1)
weather_dummies = weather_dummies.groupby(weather_dummies.index).sum()

# Объединение исходного датафрейма с преобразованными колонками
df_transform = pd.concat([df_transform, weather_dummies], axis=1)

# Удаление старой колонки weather
df_transform = df_transform.drop('weather', axis=1)

In [12]:
# road_conditions
# Преобразование колонки road_conditions с использованием метода get_dummies
road_conditions_dummies = pd.get_dummies(df_transform.road_conditions.apply(pd.Series).stack())
road_conditions_dummies.index = road_conditions_dummies.index.droplevel(-1)
road_conditions_dummies = road_conditions_dummies.groupby(road_conditions_dummies.index).sum()

# Объединение исходного датафрейма с преобразованными колонками
df_transform = pd.concat([df_transform, road_conditions_dummies], axis=1)

# Удаление старой колонки road_conditions
df_transform = df_transform.drop('road_conditions', axis=1)

#### Разделение датафрейма на обучающую и тестовую выборки (80% обучающая, 20% тестовая)

In [33]:
train_df, test_df = train_test_split(df_transform, test_size=0.2, random_state=42)

print(f"Размер обучающей выборки: {len(train_df)}")
print(f"Размер тестовой выборки: {len(test_df)}")

Размер обучающей выборки: 37695
Размер тестовой выборки: 9424


In [34]:
train_df

,severity,participants_count,region,category,datetime,light,Велосипедисты,Все участники,Дети,Мотоциклисты,...,Отсутствие элементов обустройства остановочного пункта общественного пассажирского транспорта,"Отсутствие, плохая различимость вертикальной разметки","Отсутствие, плохая различимость горизонтальной разметки проезжей части","Плохая видимость световозвращателей, размещенных на дорожных ограждениях",Плохая видимость светофора,Пыльное,Свежеуложенная поверхностная обработка,Со снежным накатом,"Сужение проезжей части, наличие препятствий, затрудняющих движение транспортных средств",Сухое
19140,0,4,Раменский,Столкновение,2016-10-04 02:05:00,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
40054,0,2,Зарайский,Столкновение,2016-08-24 12:20:00,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
18166,0,1,Рузский,Опрокидывание,2017-07-11 03:40:00,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,1
11206,0,3,Чеховский,Столкновение,2022-02-22 10:00:00,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
44572,0,4,Дмитровский,Столкновение,2017-09-25 23:55:00,2,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,0,1,Чеховский,Наезд на препятствие,2022-04-19 14:40:00,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
44732,0,4,Дмитровский,Столкновение,2017-04-23 18:50:00,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
38158,0,2,Каширский,Столкновение,2017-03-02 07:03:00,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
860,0,2,Щелковский,Наезд на препятствие,2018-05-27 17:30:00,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


#### Изменение остальных столбцов

In [35]:
# region
# Группировка данных по регионам и подсчет количества значений "С погибшими" в колонке severity
grouped_dead = train_df[train_df['severity'] == 1].groupby('region').size()
grouped_injured = 0 # train_df[train_df['severity'] == 2].groupby('region').size()

# Группировка данных по регионам и подсчет общего количества записей
total_records = train_df.groupby('region').size()

# Подсчет доли "С погибшими" для каждого региона
result = (grouped_dead + 0.5 * grouped_injured) / total_records
result = result.sort_values(ascending=False)

print(result)
average = result.mean()
print('\naverage = ' + str(average))

region_dict = result.to_dict()

# Замена значений
train_df['region'] = train_df['region'].replace(region_dict)
def replace_with_average(value):
    if value not in region_dict:
        return average
    else:
        return region_dict[value]
test_df['region'] = test_df['region'].apply(replace_with_average)

region
Воскресенский      0.308824
Волоколамский      0.273038
Можайский          0.268657
Коломенский        0.260355
Подольск           0.250000
                     ...   
Железнодорожный    0.050773
Власиха            0.043011
Дзержинский        0.037313
Котельники         0.032680
Реутов             0.028369
Length: 65, dtype: float64

average = 0.14231566105348953


In [36]:
# category
# Группировка данных по регионам и подсчет количества значений "С погибшими" в колонке severity
grouped_dead = train_df[train_df['severity'] == 1].groupby('category').size()
grouped_injured = 0 # train_df[train_df['severity'] == 2].groupby('category').size()

# Группировка данных по регионам и подсчет общего количества записей
total_records = train_df.groupby('category').size()

# Подсчет доли "С погибшими" для каждого региона
result = (grouped_dead + 0.5 * grouped_injured) / total_records
result = result.sort_values(ascending=False)

print(result)
average = result.mean()
print('\naverage = ' + str(average))

category_dict = result.to_dict()

# Замена значений
train_df['category'] = train_df['category'].replace(category_dict)
train_df['category'] = train_df['category'].fillna(average)
def replace_with_average(value):
    if value not in category_dict:
        return average
    else:
        return category_dict[value]
test_df['category'] = test_df['category'].apply(replace_with_average)
test_df['category'] = test_df['category'].fillna(average)

category
Наезд на лицо, не являющееся участником дорожного движения, осуществляющее какую-либо другую деятельность    0.428571
Падение груза                                                                                                0.363636
Наезд на лицо, не являющееся участником дорожного движения, осуществляющее производство работ                0.250000
Съезд с дороги                                                                                               0.198836
Наезд на пешехода                                                                                            0.178475
Наезд на стоящее ТС                                                                                          0.177905
Наезд на препятствие                                                                                         0.173810
Наезд на лицо, не являющееся участником дорожного движения, осуществляющее несение службы                    0.166667
Опрокидывание                                  

In [37]:
# datetime
train_df['hour'] = pd.to_datetime(train_df['datetime']).dt.hour
train_df['month'] = pd.to_datetime(train_df['datetime']).dt.month
train_df = train_df.drop('datetime', axis=1)
test_df['hour'] = pd.to_datetime(test_df['datetime']).dt.hour
test_df['month'] = pd.to_datetime(test_df['datetime']).dt.month
test_df = test_df.drop('datetime', axis=1)

In [38]:
# hour
# Группировка данных по часам и подсчет количества значений "С погибшими" в колонке severity
grouped_dead = train_df[train_df['severity'] == 1].groupby('hour').size()
grouped_injured = 0 # train_df[train_df['severity'] == 2].groupby('hour').size()

# Группировка данных по часам и подсчет общего количества записей
total_records = train_df.groupby('hour').size()

# Подсчет доли "С погибшими" для каждого часа
result = (grouped_dead + 0.5 * grouped_injured) / total_records
result = result.sort_values(ascending=False)

print(result)
average = result.mean()
print('\naverage = ' + str(average))

hour_dict = result.to_dict()

# Замена значений
train_df['hour'] = train_df['hour'].replace(hour_dict)
def replace_with_average(value):
    if value not in hour_dict:
        return average
    else:
        return hour_dict[value]
test_df['hour'] = test_df['hour'].apply(replace_with_average)

hour
2     0.259813
0     0.243827
3     0.237077
5     0.228663
4     0.228516
1     0.227151
23    0.210565
6     0.202970
21    0.188801
22    0.185905
20    0.168916
7     0.141620
18    0.140517
19    0.139037
17    0.125677
10    0.123217
15    0.119266
14    0.117983
16    0.116402
9     0.112748
11    0.107423
13    0.103132
12    0.102925
8     0.099835
dtype: float64

average = 0.1638327870669491


In [39]:
# month
# Группировка данных по регионам и подсчет количества значений "С погибшими" в колонке severity
grouped_dead = train_df[train_df['severity'] == 1].groupby('month').size()
grouped_injured = 0 # train_df[train_df['severity'] == 2].groupby('month').size()

# Группировка данных по регионам и подсчет общего количества записей
total_records = train_df.groupby('month').size()

# Подсчет доли "С погибшими" для каждого региона
result = (grouped_dead + 0.5 * grouped_injured) / total_records
result = result.sort_values(ascending=False)

print(result)
average = result.mean()
print('\naverage = ' + str(average))

month_dict = result.to_dict()

# Замена значений
train_df['month'] = train_df['month'].replace(month_dict)
def replace_with_average(value):
    if value not in month_dict:
        return average
    else:
        return month_dict[value]
test_df['month'] = test_df['month'].apply(replace_with_average)

month
10    0.163480
11    0.159937
8     0.157691
12    0.152056
7     0.150256
9     0.149250
6     0.145190
1     0.138237
3     0.136110
5     0.135101
4     0.133826
2     0.129903
dtype: float64

average = 0.1459197141455348


# Обучаем модели

In [40]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [41]:
train_df.head()

,severity,participants_count,region,category,light,Велосипедисты,Все участники,Дети,Мотоциклисты,Общ. транспорт,...,"Отсутствие, плохая различимость горизонтальной разметки проезжей части","Плохая видимость световозвращателей, размещенных на дорожных ограждениях",Плохая видимость светофора,Пыльное,Свежеуложенная поверхностная обработка,Со снежным накатом,"Сужение проезжей части, наличие препятствий, затрудняющих движение транспортных средств",Сухое,hour,month
19140,0,4,0.172185,0.118257,1,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0.259813,0.163480
40054,0,2,0.116402,0.118257,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0.102925,0.157691
18166,0,1,0.179331,0.165101,0,0,1,0,1,0,...,0,0,0,0,0,0,0,1,0.237077,0.150256
11206,0,3,0.176018,0.118257,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0.123217,0.129903
44572,0,4,0.201288,0.118257,2,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0.210565,0.149250


In [42]:
test_df.head()

,severity,participants_count,region,category,light,Велосипедисты,Все участники,Дети,Мотоциклисты,Общ. транспорт,...,"Отсутствие, плохая различимость горизонтальной разметки проезжей части","Плохая видимость световозвращателей, размещенных на дорожных ограждениях",Плохая видимость светофора,Пыльное,Свежеуложенная поверхностная обработка,Со снежным накатом,"Сужение проезжей части, наличие препятствий, затрудняющих движение транспортных средств",Сухое,hour,month
14731,0,2,0.167939,0.178475,3,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0.185905,0.150256
3732,0,3,0.150333,0.118257,3,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0.237077,0.159937
39713,0,3,0.154122,0.118257,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0.119266,0.150256
20635,1,3,0.172185,0.118257,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0.107423,0.152056
9911,0,1,0.056180,0.165101,4,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0.168916,0.133826


In [44]:
X_train = train_df.drop('severity', axis=1)
y_train = train_df['severity']
X_test = test_df.drop('severity', axis=1)
y_test = test_df['severity']

In [45]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Логистическая регрессия

In [46]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(X_train_scaled, y_train)
y_pred_logreg = logreg.predict(X_test_scaled)

print("Accuracy of Logistic Regression:", accuracy_score(y_test, y_pred_logreg))
print(classification_report(y_test, y_pred_logreg))

Accuracy of Logistic Regression: 0.851018675721562
              precision    recall  f1-score   support

           0       0.85      1.00      0.92      8026
           1       0.46      0.02      0.05      1398

    accuracy                           0.85      9424
   macro avg       0.66      0.51      0.48      9424
weighted avg       0.80      0.85      0.79      9424



### Дерево решений

In [47]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(X_train_scaled, y_train)
y_pred_dt = dt.predict(X_test_scaled)

print("Accuracy of Decision Tree:", accuracy_score(y_test, y_pred_dt))
print(classification_report(y_test, y_pred_dt))

Accuracy of Decision Tree: 0.764537351443124
              precision    recall  f1-score   support

           0       0.87      0.85      0.86      8026
           1       0.23      0.25      0.24      1398

    accuracy                           0.76      9424
   macro avg       0.55      0.55      0.55      9424
weighted avg       0.77      0.76      0.77      9424



### K-ближайших соседей

In [48]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train_scaled, y_train)
y_pred_knn = knn.predict(X_test_scaled)

print("Accuracy of K-Nearest Neighbors:", accuracy_score(y_test, y_pred_knn))
print(classification_report(y_test, y_pred_knn))

Accuracy of K-Nearest Neighbors: 0.8387096774193549
              precision    recall  f1-score   support

           0       0.86      0.97      0.91      8026
           1       0.35      0.10      0.15      1398

    accuracy                           0.84      9424
   macro avg       0.60      0.53      0.53      9424
weighted avg       0.78      0.84      0.80      9424



### Рандомный лес

In [49]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_test_scaled)

print("Accuracy of Random Forest:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

Accuracy of Random Forest: 0.8434847198641766
              precision    recall  f1-score   support

           0       0.86      0.97      0.91      8026
           1       0.39      0.10      0.15      1398

    accuracy                           0.84      9424
   macro avg       0.62      0.53      0.53      9424
weighted avg       0.79      0.84      0.80      9424



### Градиентный бустинг

In [50]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier()
gb.fit(X_train_scaled, y_train)
y_pred_gb = gb.predict(X_test_scaled)

print("Accuracy of Gradient Boosting:", accuracy_score(y_test, y_pred_gb))
print(classification_report(y_test, y_pred_gb))

Accuracy of Gradient Boosting: 0.8545203735144312
              precision    recall  f1-score   support

           0       0.86      0.99      0.92      8026
           1       0.60      0.06      0.10      1398

    accuracy                           0.85      9424
   macro avg       0.73      0.52      0.51      9424
weighted avg       0.82      0.85      0.80      9424



### Метод опорных векторов (SVM)

In [51]:
from sklearn.svm import SVC

svm = SVC()
svm.fit(X_train_scaled, y_train)
y_pred_svm = svm.predict(X_test_scaled)

print("Accuracy of Support Vector Machines:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))

Accuracy of Support Vector Machines: 0.8517614601018676
              precision    recall  f1-score   support

           0       0.85      1.00      0.92      8026
           1       0.60      0.00      0.00      1398

    accuracy                           0.85      9424
   macro avg       0.73      0.50      0.46      9424
weighted avg       0.81      0.85      0.78      9424



### Нейронная сеть (MLP)

In [52]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier()
mlp.fit(X_train_scaled, y_train)
y_pred_mlp = mlp.predict(X_test_scaled)

print("Accuracy of Multilayer Perceptron:", accuracy_score(y_test, y_pred_mlp))
print(classification_report(y_test, y_pred_mlp))

Accuracy of Multilayer Perceptron: 0.845606960950764
              precision    recall  f1-score   support

           0       0.87      0.97      0.91      8026
           1       0.44      0.14      0.21      1398

    accuracy                           0.85      9424
   macro avg       0.65      0.55      0.56      9424
weighted avg       0.80      0.85      0.81      9424



In [ ]:
from sklearn.model_selection import cross_val_score

models = [logreg, knn, dt, rf, gb, svm, mlp]
model_names = ['Logistic Regression', 'K-Nearest Neighbors', 'Decision Tree', 'Random Forest', 'Gradient Boosting', 'Support Vector Machines', 'Multilayer Perceptron']

for model, name in zip(models, model_names):
    scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='accuracy')
    print(f"{name}: {np.mean(scores)} +/- {np.std(scores)}")

Logistic Regression: 0.5278418888446742 +/- 0.004515028702613489
K-Nearest Neighbors: 0.4706459742671442 +/- 0.002965288840391264
Decision Tree: 0.4313304151744264 +/- 0.004366694564811313
Random Forest: 0.4909404430295795 +/- 0.004254862233578092
Gradient Boosting: 0.5309988062077198 +/- 0.0038693750752623658


In [ ]:
from sklearn.model_selection import GridSearchCV

# Пример настройки гиперпараметров для случайного леса:
param_grid = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

best_rf = grid_search.best_estimator_
print("Best Random Forest:", best_rf)

In [ ]:
y_pred_best_rf = best_rf.predict(X_test_scaled)
print("Accuracy of Best Random Forest:", accuracy_score(y_test, y_pred_best_rf))
print(classification_report(y_test, y_pred_best_rf))